### Engineering Database Development


In [1]:
import pandas as pd
import pyodbc
import os
import sqlalchemy.types
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer

In [2]:
# Define Database Connections

CONNAS400_PROD = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""

CONNAS400_CCSDTA = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=CCSDTA;
UID=SMY;
PWD=SMY;
"""

CONNFMA = """
Driver={FileMaker ODBC};
system=10.143.12.196;
Database=mfg8Clamps;
UID=FMODBC;
PWD=FMODBC'
"""

CONNFM = 'DSN=FM Clamp ODBC;UID=FMODBC;PWD=FMODBC'

server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)
conn_sql = engine.connect()


#### Setup Some SQL Queries

In [9]:
sql_inv = """
SELECT
    STRIP(y.itmid),
    b.qty,
    STRIP(y.itmdesc),
    STRIP(SUBSTR (Altdesc, 15, 1)) Class
FROM
    CCSDTA.DCSCIM y,
    CCSDTA.DMFCMAR x,
    CCSDTA.DCSILM b
WHERE
    x.itmid = y.itmid
    AND x.itmid = b.itmid
    AND x.plt = b.plt
    AND b.plt = '09'
    AND qty <> 0
    AND x.COSTID = 'FRZ'
    AND x.plt NOT IN ('53', '54', '55', '56', '59')
"""

sql_tbl8 = """
    SELECT Ourpart,"Band A Part Number","Housing A Part Number","Screw Part Number","Pack"
    FROM tbl8Tridon
    WHERE  (Ourpart IS NOT NULL)
        AND (RIGHT(Ourpart,1) <> '\r')
        AND ("Housing A Part Number" IS NOT NULL)
        AND (RIGHT("Housing A Part Number",1) <> '\r')
        AND ("Screw Part Number" IS NOT NULL)
        AND (RIGHT("Screw Part Number",1) <> '\r')
        AND ("Band A Part Number" IS NOT NULL)
        AND (RIGHT("Band A Part Number",1) <> '\r')
        AND ("Pack" IS NOT NULL)
        AND (RIGHT("Pack",1) <> '\r')
    ORDER BY Ourpart
    """

In [5]:
def pull_data(conn,qry):
    # Improved connection with proper error handling and connection management
    result = []
    try:
        dbcnxn = pyodbc.connect(conn, timeout=30)
        cursor = dbcnxn.cursor()
        start = timer()

        try:
            cursor.execute(qry)
            result = cursor.fetchall()
            msg = str(len(result)) + " Records Processed From Table"
        except Exception as e:
            msg = conn + ' Query Failed: ' + str(e)
            print(msg)
        finally:
            print(msg)
            print(conn + " Connect/Query Time = " + str(round((timer() - start), 3)) + " sec")

    except pyodbc.Error as e:
        msg = conn + ' Connection Failed: ' + str(e)
        print(msg)
    finally:
        if 'dbcnxn' in locals():
            dbcnxn.close()
    return result

#### Build Dataframe for Component Inventory

In [10]:
df_inv = pd.DataFrame.from_records(pull_data(CONNAS400_CCSDTA,sql_inv))
data_type_dict = {'ITMID': str, 'QTY': int, 'ITMDESC': str, 'CLASS': str}
df_inv.columns = ['ITMID', 'QTY', 'ITMDESC', 'CLASS']
df_inv = df_inv.dropna()
df_inv = df_inv.astype(data_type_dict)
df_inv = df_inv.convert_dtypes()
df_inv.sample(20)

13906 Records Processed From Table

Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=CCSDTA;
UID=SMY;
PWD=SMY;
 Connect/Query Time = 0.989 sec


,ITMID,QTY,ITMDESC,CLASS
9806,2306140,2263,BND REG 301SS .022,3
8029,6848051,1370,9/16 HG 201-201-410 SZ 048 1.5-3.5,5
9191,9990584,40,"IDEAL, LARGE DUMP BIN",2
5836,620096706,860,1/2 HG 201-201-305 SZ 096 4.5-6.5,5
6120,6260294,4800,626-02 MIC IND LAB HOME DEP @60(296004),5
10203,238031420052,650,® BORGWARNER UNIVEE T-BOLT BULK@50,5
7357,6706651,370,1/2 HG 316-316-316 SZ 006 0-0,5
1297,1278001,51613,SCR REG 305 SS,3
12763,248420425052,750,® OPTIMAS C-102408 MX BULK@50,5
1735,2013001,1124,ROD .240 CAR 1018 MIDI SCR,1


#### Build Dataframe for Clamp Data

In [15]:
print(len(result))

df_clamps = pd.DataFrame.from_records(result)
# Set Column Names
df_clamps.columns = ['PartNumber', 'Band', 'Housing', 'Screw', 'Pack']
# Set Data Types
data_type_dict = {'PartNumber': str, 'Band': str, 'Housing': str, 'Screw': str, 'Pack': str}

# Do Some Filtering and Data Cleansing
df_clamps = df_clamps[1:]
df_clamps = df_clamps.astype(data_type_dict)
df_clamps = df_clamps.dropna()
df_clamps = df_clamps.convert_dtypes()
df_clamps.sample(20)



7391


,PartNumber,Band,Housing,Screw,Pack
2587,6020052,78212,7011100,2542003,52
5830,M500152751,1208152,1209005,2374002,751
4089,671250062175,2961062,2360007,1377001MC,175
4136,671400047175,2337047,2360007,1300014MC,175
5548,M295032000,1074032,4097037,4208002,000
3154,620P96106,2745096,2351033,1278001MC,106
1199,372150080052,2745080,2360007,1278001MC,052
848,360130052052,2373052,1151006,4073003MC,052
2547,600450700052,2966712,7009900,2543001,052
4287,690055102,2292055,2360007,1278001MC,102


#### Load Inventory Data to SQL Server Table

In [11]:
data_type_dict = {'ITMID' : sqlalchemy.types.VARCHAR(255),'QTY' : sqlalchemy.types.INT,'ITMDESC' : sqlalchemy.types.VARCHAR(255),
                  'CLASS' : sqlalchemy.types.VARCHAR(255)}

df_inv.to_sql('tblCompInv', conn_sql, schema='eng', if_exists='replace', index=False, dtype=data_type_dict)

282

In [16]:
data_type_dict = {'PartNumber' : sqlalchemy.types.VARCHAR(255), 'Band' : sqlalchemy.types.VARCHAR(255),'Housing' : sqlalchemy.types.VARCHAR(255),'Screw' : sqlalchemy.types.VARCHAR(255), 'Pack' : sqlalchemy.types.VARCHAR(255)}

df_clamps.to_sql('tbl8', conn_sql, schema='eng', if_exists='replace', index=False, dtype=data_type_dict)

267